<a href="https://colab.research.google.com/github/ce888-sr/CE888_Lab/blob/main/A2/CE888_Assignment2_Model2_Weights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Activation, Dropout, Dense, Conv2D, Flatten, Dropout, ReLU, MaxPool2D, BatchNormalization, SeparableConv2D, Add,  GlobalAveragePooling2D
import cv2

In [2]:
#connect to google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
#download and unzip files
!unzip  "/content/gdrive/My Drive/Training (2).zip"
!unzip  "/content/gdrive/My Drive/Test (1).zip"

Streaming output truncated to the last 5000 lines.
  inflating: Test/Fire/resized_test_fire_frame3884.jpg  
  inflating: Test/Fire/resized_test_fire_frame3888.jpg  
  inflating: Test/Fire/resized_test_fire_frame3889.jpg  
  inflating: Test/Fire/resized_test_fire_frame3895.jpg  
  inflating: Test/Fire/resized_test_fire_frame3896.jpg  
  inflating: Test/Fire/resized_test_fire_frame39.jpg  
  inflating: Test/Fire/resized_test_fire_frame3917.jpg  
  inflating: Test/Fire/resized_test_fire_frame3920.jpg  
  inflating: Test/Fire/resized_test_fire_frame3939.jpg  
  inflating: Test/Fire/resized_test_fire_frame3943.jpg  
  inflating: Test/Fire/resized_test_fire_frame395.jpg  
  inflating: Test/Fire/resized_test_fire_frame3953.jpg  
  inflating: Test/Fire/resized_test_fire_frame3954.jpg  
  inflating: Test/Fire/resized_test_fire_frame3958.jpg  
  inflating: Test/Fire/resized_test_fire_frame3959.jpg  
  inflating: Test/Fire/resized_test_fire_frame396.jpg  
  inflating: Test/Fire/resized_test_fire_

In [4]:
train_directory = "/content/Training" #set directory to where the extracted datsets are saved
test_directory = '/content/Test'

In [5]:
#set image size and batch size
image_size = (254, 254)
batch_size = 32

In [50]:
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.1,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.8,0.9],
    validation_split=0.2)


val_datagen = ImageDataGenerator(rescale=1./255,
                                 validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1./255)

In [7]:
train_ds = train_datagen.flow_from_directory(
    train_directory,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True,
    seed = 1337,
    subset='training') # set as training data

val_ds = val_datagen.flow_from_directory(
    train_directory, # same directory as training data
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True,
    seed = 1337,
    subset='validation') # set as validation data

test_ds = test_datagen.flow_from_directory(test_directory,
                                           target_size=image_size,
                                           batch_size=batch_size,
                                           class_mode='binary',
                                           shuffle = False)

Found 31501 images belonging to 2 classes.
Found 7874 images belonging to 2 classes.
Found 8617 images belonging to 2 classes.


In [48]:
from tensorflow.keras.applications import Xception # TensorFlow ONLY
from keras.models import Model
####Investigate normal model accuracy without layers
base_model = Xception(input_shape=(254, 254, 3), weights = 'imagenet', include_top=False) #define model
x = base_model.output

# add additional layers
x = GlobalAveragePooling2D()(x) #convert features to vectors
x = Dropout(0.5)(x) # add dropout 0.5
x = Dense(2048, activation='relu')(x)
x = Dropout(0.5)(x) # add dropout 0.5
output = Dense(1, activation='sigmoid')(x) #2 classes

model = Model(base_model.input, output)

model.compile(optimizer=keras.optimizers.Adam(lr=1e-6), #low learning rate 
                  loss='binary_crossentropy',  # binary_crossentropy for binary classifier
                  metrics=['accuracy'])

#set early stopping and model weights checkpoint
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
filepath = 'a_model.hdf5'
callbacks_list = [
    EarlyStopping(monitor="val_loss", patience=10, verbose=1, restore_best_weights=True), #patience set to 5 to stop overfitting
    ModelCheckpoint(filepath, monitor ="val_loss", save_best_only=True)]

In [49]:
#train the model with the frozen layers
epochs = 40
his1 = model.fit(train_ds,
          steps_per_epoch = 100, #set epoch steps to 100
          batch_size=batch_size,
          epochs=epochs, 
          validation_data=val_ds,
          #class_weight=class_weight,
          callbacks = callbacks_list)

Epoch 1/40
100/100 [==============================] - 87s 809ms/step - loss: 0.6096 - accuracy: 0.6712 - val_loss: 0.2185 - val_accuracy: 0.9976
Epoch 2/40
100/100 [==============================] - 80s 794ms/step - loss: 0.2561 - accuracy: 0.9497 - val_loss: 0.0556 - val_accuracy: 0.9944
Epoch 3/40
100/100 [==============================] - 79s 791ms/step - loss: 0.1412 - accuracy: 0.9591 - val_loss: 0.0561 - val_accuracy: 0.9775
Epoch 4/40
100/100 [==============================] - 78s 782ms/step - loss: 0.1171 - accuracy: 0.9605 - val_loss: 0.0603 - val_accuracy: 0.9712
Epoch 5/40
100/100 [==============================] - 78s 779ms/step - loss: 0.0757 - accuracy: 0.9757 - val_loss: 0.0531 - val_accuracy: 0.9764
Epoch 6/40
100/100 [==============================] - 78s 781ms/step - loss: 0.0473 - accuracy: 0.9873 - val_loss: 0.0533 - val_accuracy: 0.9773
Epoch 7/40
100/100 [==============================] - 78s 782ms/step - loss: 0.0461 - accuracy: 0.9828 - val_loss: 0.0613 - val_ac

In [45]:
from sklearn.metrics import accuracy_score
test_ds.reset()
y_true_1 = np.asarray(test_ds.classes).astype('int32').reshape((-1, 1)) #extract labels from test set

## Test Accuracy
predictions_1 = model.predict(test_ds) #predict on test set
y_pred_1 = np.asarray(predictions_1).astype('int32').reshape((-1, 1)) #set data type to match y_true
y_pred_1 = [1 if i> 0.5 else 0 for i in predictions_1] #convert floats to classes


test_acc_1 = accuracy_score(y_true_1, y_pred_1)
print(test_acc_1)

0.6866658930022049


In [46]:
test_scores = model.evaluate(test_ds, batch_size=batch_size)

270/270 [==============================] - 26s 96ms/step - loss: 0.8884 - accuracy: 0.6867
